In [7]:
from pathlib import Path

def list_daily_station_files(root: Path = Path(r"c:\Users\Administrator\OneDrive\Documents\Github\2025_06_30"),
                             folder_name: str = "每日各站進出站人數",
                             exclude_names=("{manifest.csv}", "{schema.csv}")):
    root = Path(root)
    # 找到所有同名資料夾
    found_dirs = list(root.rglob(folder_name))
    if not found_dirs:
        print(f"找不到資料夾：{folder_name}，搜尋根目錄：{root}")
        return []

    # 正規化排除清單（小寫）
    exclude = {n.lower().strip("{}") for n in exclude_names}
    all_paths = []
    for d in found_dirs:
        files = [p.resolve() for p in d.rglob("*")
                 if p.is_file() and p.name.lower() not in exclude]
        files.sort()
        print(f"資料夾：{d}，找到 {len(files)} 個檔案（已排除 manifest/schema）")
        for f in files:
            print(f)
        all_paths.extend(files)

    # 移除重複並回傳
    unique_paths = sorted(dict.fromkeys(all_paths))
    return unique_paths

# 範例執行
paths = list_daily_station_files()
# 如果需要，paths 現在是所有絕對路徑的 list

資料夾：c:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數，找到 5 個檔案（已排除 manifest/schema）
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數20190423-20191231.csv
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數2020.csv
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數2021.csv
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數2022.csv
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數2023.csv


In [8]:
from pathlib import Path
from pprint import pprint

def list_daily_flow_files(base_dir: Path | None = None):
    """
    1. 取得 '每日各站進出站人數' 資料夾
    2. 取得其中所有 CSV 檔案的絕對路徑
    3. 排除 manifest.csv 與 schema.csv
    """
    if base_dir is None:
        base_dir = Path.cwd()  # 與 lesson19_2.ipynb 做法一致
    target_dir = base_dir / "每日各站進出站人數"
    if not target_dir.is_dir():
        raise FileNotFoundError(f"找不到資料夾: {target_dir}")

    excluded = {"manifest.csv", "schema.csv"}
    csv_paths = sorted(
        p.resolve()
        for p in target_dir.glob("*.csv")
        if p.name not in excluded
    )
    return target_dir.resolve(), csv_paths

folder_abs_path, file_abs_paths = list_daily_flow_files()

print("資料夾絕對路徑:")
print(folder_abs_path)
print("\n檔案絕對路徑 (已排除 manifest.csv, schema.csv):")
pprint([str(p) for p in file_abs_paths])

資料夾絕對路徑:
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數

檔案絕對路徑 (已排除 manifest.csv, schema.csv):
['C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數20190423-20191231.csv',
 'C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數2020.csv',
 'C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數2021.csv',
 'C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數2022.csv',
 'C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數2023.csv']


In [9]:
import pandas as pd
current_dir = Path.cwd()
# 若 CSV 與此 notebook 同資料夾
csv_path = current_dir / "台鐵車站資訊.csv"
stations_df = pd.read_csv(csv_path)
#display(stations_df.head())
stations_df = stations_df.reindex(columns=["stationCode", "stationName"])

#欄位名稱更改為[車站代碼, 車站名稱]
stations_df.columns = ["車站代碼", "車站名稱"]
stations_df

,車站代碼,車站名稱
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
238,7360,瑞芳
239,7361,海科館
240,7362,八斗子
241,7380,四腳亭


In [10]:
#建立一個function
#要concate下面迴圈的所有merged_df
def process_yearly_data(file_abs_paths, stations_df):
    merged_dfs = []
    for csv_path in file_abs_paths:
        year_df = pd.read_csv(csv_path)
        year_df.columns = ["日期", "車站代碼", "進站人數", "出站人數"]
        #display(year_df.head())
        #日期欄位目前是int64, 需要轉換為datetime格式
        year_df["日期"] = pd.to_datetime(year_df["日期"], format="%Y%m%d")
        merged_df = pd.merge(year_df, stations_df, on="車站代碼")
        merged_df = merged_df.reindex(columns=["日期","車站名稱","進站人數","出站人數"])
        merged_df.head()
        #將欄位:日期,變為index
        merged_df.set_index("日期", inplace=True)
        merged_dfs.append(merged_df)
    return pd.concat(merged_dfs)
result_df = process_yearly_data(file_abs_paths, stations_df)
result_df

,車站名稱,進站人數,出站人數
日期,,,
2019-04-23,基隆,8442,7743
2019-04-23,三坑,1394,1348
2019-04-23,八堵,2770,2423
2019-04-23,七堵,6113,6335
2019-04-23,百福,2680,2726
...,...,...,...
2023-12-31,瑞芳,7916,8252
2023-12-31,海科館,164,195
2023-12-31,八斗子,652,720


下面分步驟說明整個流程（含改善後的可直接貼回 notebook 的 function）：

1) 取得資料檔案清單
- 目的：在專案根目錄或指定 root 下遞迴尋找名為 "每日各站進出站人數" 的資料夾，列出該資料夾內所有檔案的絕對路徑。
- 要點：排除 manifest.csv、schema.csv（不分大小寫）；若有多個同名資料夾，合併所有結果並去重。

2) 讀入車站對照表
- 目的：把車站代碼 -> 車站名稱的對照讀成 DataFrame（欄位改為「車站代碼」「車站名稱」）。
- 要點：確保欄位選擇/重新命名與後續 merge 的 key 一致。

3) 處理每個年度/檔案（process_yearly_data）
- 目的：逐檔讀入 CSV、標準化欄位、解析日期、與車站對照表合併、設定索引，最後把所有檔案合併成一個 DataFrame 回傳。
- 改進重點：
  - 在 read_csv 時指定 usecols 與 names，並直接 parse_dates（減少後續轉換錯誤）。
  - 捕捉讀檔錯誤並跳過問題檔案（或記 log）。
  - 可選是否以 (日期, 車站名稱) 為 MultiIndex，避免重複索引造成分析混淆。
  - 若合併後有找不到的車站代碼，可選擇 dropna 或保留 NA（視分析需求）。

4) 常見注意事項
- 若資料量大，考慮 chunked 讀取或先處理成 parquet，提高後續速度。
- 欄位順序或欄位數不一致時，用 names+usecols 比直接重命名更穩健。
- 合併後若需要去重或彙總（例如同一日期同一站多筆），可在 concat 後使用 groupby + sum。

下面給出一個改寫過、更健壯的 process_yearly_data，可以直接放回你的 notebook（修改原有 cell）：



In [ ]:
// ...existing code...
def process_yearly_data(file_abs_paths, stations_df, *,
                        usecols=(0,1,2,3),
                        names=("日期","車站代碼","進站人數","出站人數"),
                        parse_dates=True,
                        date_format="%Y%m%d",
                        drop_missing_station=False,
                        set_multiindex=False):
    """
    逐檔讀取並處理，回傳合併後的 DataFrame。
    參數說明：
      - file_abs_paths: 可疊代 Path 物件列表
      - stations_df: 包含 '車站代碼' 與 '車站名稱' 的 DataFrame
      - drop_missing_station: True -> 合併後丟棄沒有車站名稱的列
      - set_multiindex: True -> 回傳以 (日期, 車站名稱) 的 MultiIndex
    """
    dfs = []
    for p in file_abs_paths:
        try:
            if parse_dates:
                df = pd.read_csv(
                    p,
                    usecols=usecols,
                    names=names,
                    header=0,
                    parse_dates=["日期"],
                    date_parser=lambda s: pd.to_datetime(s, format=date_format, errors="raise")
                )
            else:
                df = pd.read_csv(p, usecols=usecols, names=names, header=0)
                df["日期"] = pd.to_datetime(df["日期"].astype(str), format=date_format, errors="coerce")
            # 合併站名
            merged = pd.merge(df, stations_df, on="車站代碼", how="left")
            # 若需要丟掉沒有對應站名的資料
            if drop_missing_station:
                merged = merged.dropna(subset=["車站名稱"])
            # 重排欄位並設定索引
            merged = merged.reindex(columns=["日期","車站名稱","進站人數","出站人數"])
            if set_multiindex:
                merged.set_index(["日期","車站名稱"], inplace=True)
            else:
                merged.set_index("日期", inplace=True)
            dfs.append(merged)
        except Exception as e:
            print(f"跳過檔案 {p}，讀取或處理發生錯誤：{e}")
            continue

    if not dfs:
        return pd.DataFrame(columns=["車站名稱","進站人數","出站人數"])

    result = pd.concat(dfs)
    # 範例：若同一日期同一站有多筆，視需求可 sum()：
    # if set_multiindex: result = result.groupby(level=["日期","車站名稱"]).sum()
    return result
// ...existing code...



如需，我可以：
- 幫你把 list_daily_station_files 與 list_daily_flow_files 的邏輯統一並回傳排序好的 Path list；
- 或把整個 notebook 調整成 pipeline cell（取得檔案→讀站表→處理→儲存/匯出 parquet）。